In [14]:
import pandas as pd
import numpy as np
import copy
from pandas import *
import scipy.optimize as sopt
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import matplotlib.pyplot as plt
from itertools import permutations
from scipy.optimize import minimize_scalar, minimize, root
from scipy.optimize import bisect
np.set_printoptions(precision=3)

### Import Data

In [15]:
link_data = pd.read_excel('link_data.xlsx')
OD_matrix = pd.read_excel('O-D_matrix.xlsx')
link_data['cap'] = link_data['cap'] * 1000
#print(link_data)
#print(OD_matrix)

### Functions Defined

In [16]:
def tt_func(df):
    t0 = np.array(df.t0)
    cap = np.array(df.cap)
    x = np.array(df.x)
    return np.multiply(t0,(1+ 0.15*(np.power(np.divide(x,cap),4))))

def convergence(x_n,x_n1):
    return np.sqrt(np.sum(np.power(np.subtract(x_n1,x_n),2))) / np.sum(x_n)

def return_tt(df):
    x_n = np.array(df['x'])
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    obj =  np.sum(np.multiply(t0,np.add(x_n,0.15*np.divide(np.power(x_n,4),np.power(ca,4)))))
    return obj

def obj_func(alpha,df):
    y = np.array(df.y)
    x_n = np.array(df.x)
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    x_sub = np.add(x_n,alpha * np.subtract(y,x_n))
    obj = np.sum(np.multiply(t0,np.add(x_sub,(0.15/5)*np.divide(np.power(x_sub,5),np.power(ca,4)))))
    return obj

### Function for Dijkstra

In [17]:
def dijkstra_sp(orig,dest,df):
    G1=nx.DiGraph()
    #list_nodes=[1,2,3,4,5,6,7,8,9,10,
              # 11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    #G1.add_nodes_from(list_nodes)
    G1.add_edges_from([(link_data['O'][i],link_data['D'][i],{'weight': link_data['tt'][i]}) for i in link_data.index])
    #print(G1.edges(data=True))
    path = nx.dijkstra_path(G1, source=orig, target=dest)
    return path

In [18]:
path=dijkstra_sp(1,6,link_data)
print(path)


[1, 2, 6]


### UE 

In [19]:
### start with intitialization
## Initialization
link_data['x'] = 0
link_data['y'] = 0
Z = []
#T=[]
conv_test = 100
loop = 1
cap = link_data['cap']
while conv_test > 1e-4:
    print('Step : ', loop)
    link_data['y'] = 0
    x_n = np.array(link_data['x'])
    link_data['tt'] = tt_func(link_data)
    for i in range (len(OD_matrix)):
        orig=OD_matrix['O'][i]
        dest=OD_matrix['D'][i]
        fl=OD_matrix['Q'][i]
        path=dijkstra_sp(orig,dest,link_data)
        for k in range(len(path)-1):
            o_new = path[k] 
            d_new = path[k+1]
            link_data.loc[(link_data.O==o_new) & (link_data.D==d_new),'y'] +=fl
    if loop!= 1:
        alpha = minimize_scalar(obj_func,args=(link_data), bounds=(0.0, 1.0),method='bounded').x
    else :
        alpha = 1
    print("Alpha : ",alpha)
    print("Z : ",obj_func(alpha,link_data))
    Z.append(obj_func(alpha,link_data))
    y = np.array(link_data['y'])
    #print(x_n,y)
    x_n1 = x_n + alpha * (y-x_n)
    link_data['x'] = x_n1 ## Update x to update t
    #print("T : ",return_tt(link_data))
    #T.append(return_tt(link_data))
    if loop != 1:
        conv_test= convergence(x_n,x_n1)
        print("Conv : ",conv_test)
    loop += 1
link_data['ratio'] = link_data['x'] / link_data['cap']
print(link_data)
        

Step :  1
Alpha :  1
Z :  1191893.2628928602
Step :  2
Alpha :  0.36175784724044485
Z :  1211664.262124838
Conv :  0.016114173589425004
Step :  3
Alpha :  0.9999940391390134
Z :  1207635.7389367851
Conv :  0.01755344054192424
Step :  4
Alpha :  0.12810051568630998
Z :  1207479.8114731915
Conv :  0.004331868605226641
Step :  5
Alpha :  0.08391508353161287
Z :  1207461.8336773503
Conv :  0.00163625979033149
Step :  6
Alpha :  0.4997745666139474
Z :  1207352.9884236637
Conv :  0.004264160838276197
Step :  7
Alpha :  0.0720118368911302
Z :  1207331.6681080768
Conv :  0.0016707935363367908
Step :  8
Alpha :  0.46057216730328115
Z :  1207284.5995861099
Conv :  0.0027086503628511163
Step :  9
Alpha :  0.0707709489929555
Z :  1207262.726321677
Conv :  0.0016074294680648012
Step :  10
Alpha :  0.3325043428989475
Z :  1207246.0759569888
Conv :  0.0016759859170538648
Step :  11
Alpha :  0.0429229899778178
Z :  1207236.5815773548
Conv :  0.0010129011225845543
Step :  12
Alpha :  0.0131624228603206

In [29]:
# for i in range (len(OD_matrix)):
#         orig=OD_matrix['O'][i]
#         dest=OD_matrix['D'][i]
#         fl=OD_matrix['Q'][i]
#         print(orig,dest,fl)